In [1]:
from transformers import AutoModelForMaskedLM, AutoModelForSequenceClassification,AutoTokenizer
import torch
from datasets import load_dataset,concatenate_datasets,Dataset
import math
from torch.utils.data import DataLoader
from transformers import default_data_collator
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import argparse
import sys
import os
sys.path.append(os.path.abspath('../'))
import preprocessing
import preprocessing
import pickle
from sklearn.preprocessing import LabelEncoder
from transformers import get_linear_schedule_with_warmup
import random
import re
import numpy as np
import pandas as pd

In [2]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
model_checkpoint = "KBLab/bert-base-swedish-cased"
model =  AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at KBLab/bert-base-swedish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model_finetuned = AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2175500")
model_finetuned=model_finetuned.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2175500 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model_exbert=  AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/exbert-finetuned-imdb/checkpoint-6054000")
model_exbert=model_exbert.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/laurinemeier/swerick/exbert-finetuned-imdb/checkpoint-6054000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
exbert_tokenizer = AutoTokenizer.from_pretrained("/home/laurinemeier/swerick/exbert_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from transformers import PreTrainedTokenizerFast

swerick_tokenizer= PreTrainedTokenizerFast(
    tokenizer_file="/home/laurinemeier/swerick/alvis_project/pretraining_from_scratch/tokenizer_swerick.json",
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)

In [6]:
swerick_tokenizer.save_pretrained("swerick_tokenizer")

('swerick_tokenizer/tokenizer_config.json',
 'swerick_tokenizer/special_tokens_map.json',
 'swerick_tokenizer/tokenizer.json')

In [ ]:
import transformers
config = transformers.BertConfig.from_pretrained("/home/laurinemeier/swerick/alvis_project/pretraining_from_scratch/checkpoint-5258900")
mosaicBert = AutoModelForMaskedLM.from_pretrained("/home/laurinemeier/swerick/alvis_project/pretraining_from_scratch/checkpoint-5258900",config=config,trust_remote_code=True)

In [9]:
tokenizer= AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
def filter_NaN(subset,example):
    return example[subset] is not None


In [4]:
def subset_label(dataset,nb_obs,label_name):
    df = dataset.to_pandas()

    # Calculer le nombre d'observations pour chaque étiquette
    grouped_data = df.groupby(label_name)

    # Calculer le nombre d'observations par étiquette pour obtenir une répartition uniforme
    total_samples = nb_obs
    samples_per_label = total_samples // len(grouped_data.groups)

    # Créer une liste pour stocker les observations échantillonnées
    sampled_data = []

    # Prélever aléatoirement les observations pour chaque groupe de label
    for group_label, group_data in grouped_data.groups.items():
        group_dataset=dataset.select(group_data)
        label_data = group_dataset.shuffle(seed=np.random.randint(1, 1000)).select(range(min(len(group_data), samples_per_label)))
        sampled_data.extend(label_data)

    # Mélanger les observations pour obtenir un ordre aléatoire
    np.random.shuffle(sampled_data)

    # Créer un Dataset Hugging Face à partir des observations échantillonnées
    sampled_dataset = Dataset.from_dict({key: [example[key] for example in sampled_data] for key in sampled_data[0]})
    
    return sampled_dataset

In [5]:
def subset_random(dataset,nb_ob):
    dataset=dataset.shuffle()
    echantillon_aleatoire = dataset.select(range(nb_ob))
    return echantillon_aleatoire

In [6]:
def tokenize_function(examples):
    result = tokenizer(examples["Note"],padding=True, truncation=True,max_length=512)
    return result

In [7]:
def evaluate(model, loader):
    loss, accuracy = 0.0, []
    model.eval()
    for batch in tqdm(loader, total=len(loader)):
        input_ids = batch["input_ids"].to(device)
        input_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        output = model(input_ids,
            token_type_ids=None, 
            attention_mask=input_mask, 
            labels=labels)
        loss += output.loss.item()
        preds_batch = torch.argmax(output.logits, axis=1)
        batch_acc = torch.mean((preds_batch == labels).float())
        accuracy.append(batch_acc)
        
    accuracy = torch.mean(torch.tensor(accuracy))
    return loss, accuracy


In [8]:
def extract_date_from_filename(protocole):
    match = re.search(r'/(\d+)/', protocole)
    if match:
        year = match.group(1)
        return int(year[:4])
    else:
        return None

In [9]:
data_files = {"train": "swerick_data_party_train.pkl", "test": "swerick_data_party_test.pkl"}
party_dataset = load_dataset("pandas",data_files=data_files)
print(party_dataset)

DatasetDict({
    train: Dataset({
        features: ['protocole', 'Note', 'id', 'party', 'gender'],
        num_rows: 3378877
    })
    test: Dataset({
        features: ['protocole', 'Note', 'id', 'party', 'gender'],
        num_rows: 725974
    })
})


In [7]:
df=pd.read_pickle("swerick_data_party_train.pkl")
df= df.sample(n=100, random_state=42)
df.to_csv("swerick_data_party_train_100.csv")

In [ ]:
from collections import Counter
# Fonction pour mettre à jour le Counter basé sur le champ 'party'
def add_party_counts(batch, counts):
    print("hpp")
    counts.update(batch['party'])
    return batch  


counts = Counter()
party_dataset["train"].map(add_party_counts, fn_kwargs={'counts': counts}, batched=True)

print(counts)

In [ ]:
counts

In [ ]:
parties_to_keep = {party for party, count in counts.items() if count >=100 and party !='None'}
print(parties_to_keep)

party_dataset = party_dataset.filter(lambda example: example['party'] in parties_to_keep)



In [ ]:
print(party_dataset['train'])

In [ ]:
party_alignment = {
    '"vänstern"':'left',
    'Folkpartiet':'left',
    'Folkpartiet (1895–1900)':'left',
    'Frisinnade folkpartiet':'left',
    'Kommunistiska partiet':'left',
    'Socialdemokraterna':'left',
    'Socialdemokratiska vänstergruppen':'left',
    'Socialistiska partiet':'left',
    'Miljöpartiet':'agro',
    'Vänsterpartiet' :'left',
    'Sveriges kommunistiska parti': 'left',
    'Andra kammarens frihandelsparti' :'liberal',
    'Första kammarens minoritetsparti':'right',
    'Ehrenheimska partiet':'right',
    'Första kammarens moderata parti':'liberal',
    'Bondeförbundet':'agro',
    'Centerpartiet' :'liberal',
    'Det förenade högerpartiet':'right',
    'Första kammarens ministeriella grupp':'right',
    'Första kammarens nationella parti':'right',
    'Första kammarens protektionistiska parti':'right',
    'högervilde':'right',
    'Högerpartiet':'right',
    'Högerpartiet de konservativa':'right',
    'Kilbomspartiet':'left',
    'Lantmannapartiets filial' :'agro',
    'Nya lantmannapartiet':'agro',
    'Skånska partiet':'agro',
    'Högerns riksdagsgrupp':'right',
    'Högerpartiet ':'right',
    'Jordbrukarnas fria grupp':'agro',
    'borgmästarepartiet':'right',
    'Junkerpartiet':'right',
    'Kristdemokraterna' :'right',
    'Moderaterna' :'right',
    'Sverigedemokraterna' :'right',
    'Nationella framstegspartiet':'right',
    'Ny demokrati' :'right',
    'Gamla lantmannapartiet':'agro',
    'Lantmannapartiet':'agro',
    'Andra kammarens center':'liberal',
    'frihandelsvänlig vilde':'liberal',
    'frisinnad vilde':'liberal',
    'Stockholmsbänken':'liberal',
    'Frihandelsvänliga centern':'liberal',
    'ministeriella partiet':'liberal',
    'Centern (partigrupp 1873-1882)':'liberal',
    'Centern (partigrupp 1885-1887)':'liberal',
    'Första kammarens konservativa grupp':'right',
    'Frisinnade landsföreningen':'liberal',
    'Frisinnade försvarsvänner':'liberal',
    'Friesenska diskussionsklubben':'liberal',
    'Liberala riksdagspartiet':'liberal',
    'Liberala samlingspartiet':'liberal',
    'Liberalerna':'liberal',
    'Nya centern (partigrupp 1883-1887)':'liberal',
    'Nyliberala partiet':'liberal',
    'Medborgerlig samling (1964–1968)':'liberal',
    'Lantmanna- och borgarepartiet inom andrakammaren':'agro'
}

def determine_alignment(example):
    party = example['party']
    alignment = party_alignment.get(party, 'Non classifié')
    example['alignment'] = alignment
    return example

In [ ]:
party_dataset = party_dataset.map(determine_alignment)

In [ ]:
party_dataset

In [ ]:
set(list(parties['train']['party']))

In [13]:
data_files = {"train": "/home/laurinemeier/swerick/evaluation/swerick_data_seg_train.pkl", "test": "/home/laurinemeier/swerick/evaluation/swerick_data_seg_test.pkl"}
seg_dataset = load_dataset("pandas",data_files=data_files)
print(seg_dataset)

DatasetDict({
    train: Dataset({
        features: ['Text', 'Note/seg', '__index_level_0__'],
        num_rows: 11286588
    })
    test: Dataset({
        features: ['Text', 'Note/seg', '__index_level_0__'],
        num_rows: 2821648
    })
})


In [10]:
dates = [extract_date_from_filename(row['protocole']) for row in party_dataset['train']]
dates_test = [extract_date_from_filename(row['protocole']) for row in party_dataset['test']]
party_dataset['train'] = party_dataset['train'].add_column('date', dates)
party_dataset['test'] = party_dataset['test'].add_column('date', dates_test)

print(party_dataset["train"][0])
print(party_dataset["test"][0])

{'protocole': 'data/1867/prot-1867--ak--0119.xml', 'Note': 'För min del kan jag ej se något hinder för företagande af dessa val ännu något tidigare än Herr Talmannen föreslagit. Jag har föreställt mig, att vi så mycket som möjligt borde taga vara på den dyrbara tiden, och att nu ifrågakomna val kunde förrättas t. ex. på Tisdagen, och valen till suppleanter i Utskotten på Thorsdagen.', 'id': 'i-PyY1Vo1W6WaajphhpnKHN8', 'party': None, 'gender': 'man', 'date': 1867}
{'protocole': 'data/1867/prot-1867--ak--0118.xml', 'Note': 'Mine Herrar! Sannolikt är det för många bland Eder oväntadt att se en prest intaga talmansstolen i Riksdagens Andra Kammare. Det sker också icke efter min egen önskan. Jag har lika litet eftersträfvat detta höga och ansvarsfulla förtroende, som jag sökt sjelfva riksdagsmannakallets hedrande uppdrag. Men då vahnännen inom den krets jag tillhör lemnat mig det sednare, och Kongl. Maj:t sedermera behagat förläna mig det förra, har jag ej haft giltiga skäl att undandraga m

In [ ]:
def determine_reform(example):
    example['reform'] = 'pre' if example['date'] <= 1912 else 'post'
    return example

# Appliquer la fonction à chaque ligne du dataset
party_dataset = party_dataset.map(determine_reform)

In [11]:
def year_filter(example, year):
    print("hey")
    return example['date'] >= year

party_dataset = party_dataset.filter(lambda x: year_filter(x, 2000))
party_dataset

DatasetDict({
    train: Dataset({
        features: ['protocole', 'Note', 'id', 'party', 'gender', 'date'],
        num_rows: 1322959
    })
    test: Dataset({
        features: ['protocole', 'Note', 'id', 'party', 'gender', 'date'],
        num_rows: 281978
    })
})

In [12]:
print(set(list(party_dataset["train"]["date"])))

{2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021}


In [ ]:
data_files = {"valid": "swerick_data_party_valid.pkl"}
party_valid_dataset = load_dataset("pandas",data_files=data_files)
print(party_valid_dataset)

In [ ]:
data_files = {"valid": "swerick_data_intro_valid.csv"}
party_valid_dataset = load_dataset("pandas",data_files=data_files)
print(party_valid_dataset)

In [ ]:
dates_valid = [extract_date_from_filename(row['protocole']) for row in party_valid_dataset['valid']]
party_valid_dataset['valid'] = party_valid_dataset['valid'].add_column('date', dates_valid)

In [11]:
from sklearn.preprocessing import StandardScaler

date_scaler = StandardScaler()
dates_train_2d = [[date] for date in party_dataset['train']["date"]]
dates_test_2d=[[date] for date in party_dataset['test']["date"]]
date_scaler.fit(dates_train_2d)
dates_train =date_scaler.transform(dates_train_2d)
dates_test =date_scaler.transform(dates_test_2d)
print(dates_train)
party_dataset['train'] = party_dataset['train'].add_column('date_scaled', dates_train.squeeze())
party_dataset['test'] = party_dataset['test'].add_column('date_scaled', dates_test.squeeze())


[[-2.57723263]
 [-2.57723263]
 [-2.57723263]
 ...
 [ 1.05391207]
 [ 1.05391207]
 [ 1.05391207]]


In [10]:
party_dataset["train"]=party_dataset["train"].filter(lambda x : filter_NaN("gender",x))
party_dataset["test"]=party_dataset["test"].filter(lambda x : filter_NaN("gender",x))

In [16]:
def filter_notes(example):
    note = example['Note']
    return ('Herr' in note or
            'Fru'  in note)
    
    
party_dataset_note_train=party_dataset['train'].filter(filter_notes)
party_dataset_note_test=party_dataset['test'].filter(filter_notes)
print(party_dataset['train'])

Filter:   0%|          | 0/1322822 [00:00<?, ? examples/s]

Filter:   0%|          | 0/281935 [00:00<?, ? examples/s]

Dataset({
    features: ['protocole', 'Note', 'id', 'party', 'gender', 'date'],
    num_rows: 1322822
})


In [17]:
print(party_dataset_note_train[:10]['Note'])

['Fru talman! Vi har anledning att vara stolta över', 'Fru talman! Sverige har ett rikt kulturliv. Det ska vi värna om och vara stolta över.', 'Fru talman! Sverige får aldrig någonsin mer hamna i den tvångströja som växande underskott sätter på politiken.', 'Fru talman! Ärade ledamöter och åhörare! Det är med stor stolthet som jag i dag kan presentera en budget som kännetecknas av reformer och ett fortsatt ansvar för en balanserad ekonomisk utveckling. Inflationen är låg. Räntorna är låga. De offentliga finanserna uppvisar betydande överskott. Redan nästa år har nettoskulden försvunnit. Vår ekonomi växer i år med nära 4 % och förväntas öka nästan lika mycket nästa år. Målet om 4 % arbetslöshet, som sattes upp för fyra år sedan, är så gott som nått. I dagens situation har vi utrymme både för att öka utgifterna och att sänka skatterna, och ändå kan vi betala av mer på statsskulden än vi tidigare hade planerat.', 'Fru talman! Den första meningen i budgeten säger att detta är en budget för

In [18]:
def filter_size(example):
    note = example['Note']
    word_count = len(note.split())
    return word_count > 30

party_dataset_note_train=party_dataset_note_train.filter(filter_size)
party_dataset_note_test=party_dataset_note_test.filter(filter_size)
print(party_dataset_note_train)

Filter:   0%|          | 0/267159 [00:00<?, ? examples/s]

Filter:   0%|          | 0/57457 [00:00<?, ? examples/s]

Dataset({
    features: ['protocole', 'Note', 'id', 'party', 'gender', 'date'],
    num_rows: 199990
})


In [ ]:
party_valid_dataset["valid"]=party_valid_dataset["valid"].filter(lambda x : filter_NaN("party",x))

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(party_dataset["train"]["gender"])
label_names = label_encoder.classes_
label_dict={ i : label_names[i] for i in  range(len(label_names))}
print(label_dict)

In [ ]:
print(label_names)

In [ ]:
with open("labels_gender.pkl", "wb") as fp:   
   pickle.dump(label_names, fp)

In [ ]:
with open("labels.pkl","rb") as f :
    label_names=pickle.load(f)

print(label_names.tolist())

In [ ]:
party_dataset["train"]=party_dataset["train"].map(lambda example :{"party_labels" : label_encoder.transform([example["party"]])[0]})
party_dataset["test"]=party_dataset["test"].map(lambda example :{"party_labels" : label_encoder.transform([example["party"]])[0]})

In [ ]:
party_valid_dataset["valid"]=party_valid_dataset["valid"].map(lambda example :{"party_labels" : label_encoder.transform([example["party"]])[0]})

In [ ]:
len(set(list(party_dataset_year['train']['party'])))

In [17]:
party_train_datasets = subset_label(party_dataset_year["train"],10000,"party")
party_test_datasets = subset_label(party_dataset["test"],5000,"party")
#party_valid_datasets = subset_label(party_valid_dataset["valid"],5000,"party")

In [14]:
train_set1 = subset_random(party_dataset["train"],100)
train_set2 = subset_random(party_dataset["train"],200)
train_set3 = subset_random(party_dataset["train"],500)
train_set4= subset_random(party_dataset["train"],1000)
test_set = subset_random(party_dataset["test"],10000)


In [12]:
test_set = subset_random(party_dataset['test'],5000)
train_set = subset_random(party_dataset["train"],1000)

In [ ]:
print(party_train_datasets)
print(party_test_datasets)
print(party_valid_datasets)

In [ ]:
party_dataset = concatenate_datasets([train_set,test_set,valid_set])

In [ ]:
tokenized_train_datasets = train_set.map(tokenize_function,batched=True )
tokenized_test_datasets = test_set.map(tokenize_function,batched=True )
tokenized_valid_datasets = valid_set.map(tokenize_function,batched=True )
tokenized_train_datasets

In [ ]:
tokenized_train_datasets=tokenized_train_datasets.remove_columns(["protocole","id","party","gender","Note"])
tokenized_test_datasets=tokenized_test_datasets.remove_columns(["protocole","id","party","gender","Note"])
tokenized_valid_datasets=tokenized_valid_datasets.remove_columns(["protocole","id","party","gender","Note"])

In [ ]:
tokenized_train_datasets=tokenized_train_datasets.rename_column("party_labels","labels")
tokenized_test_datasets=tokenized_test_datasets.rename_column("party_labels","labels")
tokenized_valid_datasets=tokenized_valid_datasets.rename_column("party_labels","labels")





In [ ]:
tokenized_train_datasets.set_format(type="torch",columns=["input_ids","labels","attention_mask"])
tokenized_test_datasets.set_format(type="torch",columns=["input_ids","labels","attention_mask"])
tokenized_valid_datasets.set_format(type="torch",columns=["input_ids","labels","attention_mask"])

In [ ]:
batch_size = 64
num_workers=4

train_loader = DataLoader(
        tokenized_train_datasets,
        shuffle=True,
        batch_size = batch_size,
        num_workers = num_workers
    )

valid_loader = DataLoader(
        tokenized_valid_datasets,
        shuffle=False,
        batch_size = batch_size,
        num_workers = num_workers
    )

# Not used atm
test_loader = DataLoader(
        tokenized_test_datasets,
        shuffle=False,
        batch_size = batch_size,
        num_workers = num_workers
    )

In [ ]:
n_epochs =10
model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=len(label_dict),
        id2label=label_dict).to("cpu")

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)
num_training_steps = len(train_loader) * n_epochs
num_warmup_steps = num_training_steps // 10

# Linear warmup and step decay
scheduler = get_linear_schedule_with_warmup(
    optimizer = optimizer,
    num_warmup_steps = num_warmup_steps,
    num_training_steps = num_training_steps
    )

In [ ]:
import pandas as pd

df=pd.read_pickle("swerick_data_party_train.pkl")
df = df.rename(columns={"Note":"content","party" : "tag"})
df.to_csv("swerick_data_party_train.csv")

In [ ]:
def label_party(example):
    if example =='left':
        return 0
    elif example =="center":
        return 1
    elif example =="right":
        return 2

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("swerick_subsetdata_alignment_test_withoutcenter.csv")
df["tag"][30]

In [ ]:
df =df.dropna(subset="tag")
df.to_csv("swerick_data_party_train.csv")

In [15]:
#Count the number of words in the "Note" column
df =party_dataset["train"].to_pandas()

# Rename the columns
\
# Save the filtered DataFrame to a CSV file
df.to_csv("swerick_subsetdata_date_train.csv", index=False)

In [4]:
!python3 compare_models.py --model_filename1 "trained_party_classification-" --model_filename2 "trained_party_classification+061000" --data_path "/home/laurinemeier/swerick/evaluation/swerick_subsetdata_party_test.csv"


None
8
{0: 'Centerpartiet', 1: 'Kristdemokraterna', 2: 'Liberalerna', 3: 'Miljöpartiet', 4: 'Moderaterna', 5: 'Socialdemokraterna', 6: 'Sverigedemokraterna', 7: 'Vänsterpartiet'}
  0%|                                                  | 0/1875 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already be

In [2]:
! python3 train_binary_bert_base.py --model_filename "trained_party_classification"+"061000" --base_model "/home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2061000" --data_path "/home/laurinemeier/swerick/evaluation/swerick_subsetdata_party_train.csv"

14:20:27 [INFO] (train-bert): Load and save tokenizer...
14:20:28 [INFO] (train-bert): Preprocess datasets...
14:21:17 [INFO] (train-bert): Labels: tensor([4, 2, 5,  ..., 4, 5, 4])
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2061000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
14:21:19 [TRAIN] (train-bert): Epoch 0 starts!
100%|███████████████████████████████████████| 1250/1250 [02:08<00:00,  9.74it/s]
14:41:56 [TRAIN] (train-bert): Training Loss: 26.608
14:41:56 [TRAIN] (train-bert): Validation Loss: 23.982
14:41:56 [TRAIN] (train-bert): Validation accuracy: 0.4438999891281128
14:41:56 [INFO] (train-bert): Best validation loss so far
14:41:57 [TRAIN] (train-bert):

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))
import preprocessing
from evaluation import party_classifaction_scratch,reform_classification,reform_scratch_classfication,party_classifaction_bis
from evaluation import regression_year
from evaluation import party_gender_detection
from evaluation import train_binary_bert,compare_model
reform_scratch_classfication("trained_party_classification","KBLab/bert-base-swedish-cased","/home/laurinemeier/swerick/exbert-finetuned-imdb/checkpoint-6054000","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_party_train.csv","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_party_test.csv",tokenizer2="/home/laurinemeier/swerick/exbert_tokenizer")
 #party_gender_detection("/home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2175500")
#regression_year("/home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2175500","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_date_train100.csv")
#reform_classification("/home/laurinemeier/swerick/finetuning_hugging_whitespace_bis-finetuned-imdb/checkpoint-2175500","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_reform_train.csv","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_reform_test.csv")
#reform_scratch_classfication("/home/laurinemeier/swerick/exbert-finetuned-imdb/checkpoint-3511320","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_reform_train.csv","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_reform_test.csv","/home/laurinemeier/swerick/exbert_tokenizer")

Party alignement classification
training
stdout: 10:57:36 [INFO] (train-bert): Load and save tokenizer...
10:57:37 [INFO] (train-bert): Preprocess datasets...
10:58:27 [INFO] (train-bert): Labels: tensor([4, 2, 5,  ..., 4, 5, 4])
10:58:28 [TRAIN] (train-bert): Epoch 0 starts!
11:19:08 [TRAIN] (train-bert): Training Loss: 28.224
11:19:08 [TRAIN] (train-bert): Validation Loss: 25.669
11:19:08 [TRAIN] (train-bert): Validation accuracy: 0.40685001015663147
11:19:08 [INFO] (train-bert): Best validation loss so far
11:19:08 [TRAIN] (train-bert): Epoch 1 starts!
11:39:50 [TRAIN] (train-bert): Training Loss: 23.514
11:39:50 [TRAIN] (train-bert): Validation Loss: 24.249
11:39:50 [TRAIN] (train-bert): Validation accuracy: 0.44475001096725464
11:39:50 [INFO] (train-bert): Best validation loss so far
11:39:51 [TRAIN] (train-bert): Epoch 2 starts!
12:00:34 [TRAIN] (train-bert): Training Loss: 18.707
12:00:34 [TRAIN] (train-bert): Validation Loss: 25.496
12:00:34 [TRAIN] (train-bert): Validation acc

In [ ]:
!python3 compare_models.py --model_filename1 "trained_alignment_classification_good" --model_filename2 "trained_alignment_party_good_118250" --data_path "/home/laurinemeier/swerick/evaluation/swerick_subsetdata_alignment_test_withoutcenter.csv"

In [ ]:
!python3 compare_models.py --model_filename1 "trained_alignment_classification_good" --model_filename2 "trained_alignment_party_good_118250" --data_path "/home/laurinemeier/swerick/evaluation/swerick_subsetdata_alignment_test_withoutcenter.csv"

In [ ]:
!python3 train_binary_bert.py --model_filename1 "trained_alignment_classification"  --model_filename2 "trained_alignment_party_good_118250" --data_path "/home/laurinemeier/swerick/evaluation/swerick_subsetdata_alignment_train_withoutcenter.csv" --learning_rate 2e-5

In [ ]:
with open("comparison_results.txt", "a") as file:
        file.write(f"{best_settings },{best_accuracy}\n")

In [14]:
from evaluation import regression_year,reform_scratch_classfication

reform_scratch_classfication("trained_reform_classification","KBLab/bert-base-swedish-cased","/home/laurinemeier/swerick/exbert-finetuned-imdb/checkpoint-6054000","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_reform_train.csv","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_reform_test.csv",tokenizer2="/home/laurinemeier/swerick/exbert_tokenizer")

Party alignement classification
training
stdout: 10:47:32 [INFO] (train-bert): Load and save tokenizer...
10:47:33 [INFO] (train-bert): Preprocess datasets...
10:47:33 [INFO] (train-bert): Labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
   

In [37]:
class LIMExplainer:

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = "cpu"
        

    def predict(self, data):
        self.model.to(self.device)

        ref = self.tokenizer.tokenize(data[0])
        data_temp = [ref]
        for x in data[1:]:
            ref_temp = ref.copy()
            new = ["" for _ in range(len(ref))]
            x = self.tokenizer.tokenize(x)
            #if ".   ." in x:
            #    x = [xx if xx != ".   ." else [xxx for xxx in xx.split(" ") if xxx] for xx in x]
            #    x = [xx for xxx in x for xx in xxx]
            for w in x:
                id = ref_temp.index(w)
                new[id] = w
                ref_temp[id] = ""
            data_temp.append(new)

        data_temp = [["[PAD]" if xx == "" else xx for xx in x] for x in data_temp]
        data_temp = [" ".join(x) for x in data_temp]

        tokenized_nopad = [self.tokenizer.tokenize(text) for text in data_temp]
        MAX_SEQ_LEN = max(len(x) for x in tokenized_nopad)
        tokenized_text = [["[PAD]", ] * MAX_SEQ_LEN for _ in range(len(data))]
        for i in range(len(data)):
            tokenized_text[i][0:len(tokenized_nopad[i])] = tokenized_nopad[i][0:MAX_SEQ_LEN]
        indexed_tokens = [self.tokenizer.convert_tokens_to_ids(tt) for tt in tokenized_text]
        tokens_tensor = torch.tensor(indexed_tokens)
        tokens_tensor = tokens_tensor.to(self.device)
        with torch.no_grad():
            outputs = self.model(input_ids=tokens_tensor)
            print(outputs)
            # logits = outputs[0]
            predictions = outputs.logits.detach().cpu().numpy()
        final = [self.softmax(x) for x in predictions]
        return np.array(final)

    def softmax(self, it):
        exps = np.exp(np.array(it))
        return exps / np.sum(exps)

    def split_string(self, string):
        data_raw = self.tokenizer.tokenize(string)
        data_raw = [x for x in data_raw if x not in ".,:;'"]
        return data_raw

In [38]:
import shap
from lime.lime_text import LimeTextExplainer

def contains_old_spelling(text):
    old_spelling_patterns = [r'\bdt\b', r'\bfv\b', r'\bf\b', r'\bw\b', r'\bdh\b', r'\bgh\b']
    for pattern in old_spelling_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return True
    return False

model_exbert_classification = AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/evaluation/trained_reform_classification054000")
df=pd.read_csv("/home/laurinemeier/swerick/evaluation/trained_reform_classification054000/missclassifed_exbert_KBBERT.csv")
sample_texts = df['content'].tolist()
predictor = LIMExplainer(model_exbert_classification, exbert_tokenizer)
label_names = ["0", "1"]
explainer = LimeTextExplainer(class_names=label_names, split_expression=predictor.split_string)
temp = predictor.split_string(sample_texts[0])
xp = explainer.explain_instance(text_instance=sample_texts[0], classifier_fn=predictor.predict, num_features=len(temp))
words = exp.as_list()
exp.show_in_notebook(text=True, labels=(exp.available_labels()[0],))

In [15]:
from sklearn.metrics import classification_report

data = {
    'content': [
        'hvad', 'hvar', 'hvarje', 'hvem', 'hvilka',
        'vad', 'var', 'varje', 'vem', 'vilka',
        'hvardag', 'vardag', 'hvarifrån', 'varifrån', 'hvarhelst',
        'varhelst', 'hvarken', 'varken', 'hvarmed', 'varmed',
        'hvarom', 'varom', 'hvarpå', 'varpå', 'hvarifrån',
        'varifrån', 'hvarvid', 'varvid', 'hvarest', 'varest',
        'hvarjehanda', 'varjehanda', 'hvart', 'vart', 'hvartannat',
        'vartannat', 'hvarannan', 'varannan', 'hvarvidlag', 'varvidlag',
        'hvarenda', 'varenda', 'hvarken', 'varken', 'hvarifrån',
        'varifrån', 'hvarpå', 'varpå', 'hvarvid', 'varvid',
        'hvarom', 'varom', 'hvarhelst', 'varhelst', 'hvarifrån',
        'varifrån', 'hvarmed', 'varmed', 'hvarest', 'varest',
        'hvarjehanda', 'varjehanda', 'hvart', 'vart', 'hvartannat',
        'vartannat', 'hvarannan', 'varannan', 'hvarvidlag', 'varvidlag',
        'hvarenda', 'varenda', 'hvarken', 'varken', 'hvad',
        'vad', 'hvar', 'var', 'hvarje', 'varje',
        'hvem', 'vem', 'hvilka', 'vilka', 'hvardag',
        'vardag', 'hvarifrån', 'varifrån', 'hvarhelst', 'varhelst',
        'hvarken', 'varken', 'hvarmed', 'varmed', 'hvarom',
        'varom', 'hvarpå', 'varpå', 'hvarifrån', 'varifrån',
        'hvarvid', 'varvid', 'hvarest', 'varest', 'hvarjehanda',
        'varjehanda', 'hvart', 'vart', 'hvartannat', 'vartannat',
        'hvarannan', 'varannan', 'hvarvidlag', 'varvidlag', 'hvarenda',
        'varenda', 'hvarken', 'varken', 'hvarifrån', 'varifrån',
        'hvarpå', 'varpå', 'hvarvid', 'varvid', 'hvarom',
        'varom', 'hvarhelst', 'varhelst', 'hvarifrån', 'varifrån',
        'hvarmed', 'varmed', 'hvarest', 'varest', 'hvarjehanda',
        'varjehanda', 'hvart', 'vart', 'hvartannat', 'vartannat',
        'hvarannan', 'varannan', 'hvarvidlag', 'varvidlag', 'hvarenda',
        'varenda', 'hvarken', 'varken', 'hvad', 'vad',
        'hvar', 'var', 'hvarje', 'varje', 'hvem',
        'vem', 'hvilka', 'vilka', 'hvardag', 'vardag',
        'hvarifrån', 'varifrån', 'hvarhelst', 'varhelst', 'hvarken',
        'varken', 'hvarmed', 'varmed', 'hvarom', 'varom',
        'hvarpå', 'varpå', 'hvarifrån', 'varifrån', 'hvarvid',
        'varvid', 'hvarest', 'varest', 'hvarjehanda', 'varjehanda',
        'hvart', 'vart', 'hvartannat', 'vartannat', 'hvarannan',
        'varannan', 'hvarvidlag', 'varvidlag', 'hvarenda', 'varenda',
        'hvarken', 'varken', 'hvad', 'vad', 'hvar',
        'var', 'hvarje', 'varje', 'hvem', 'vem',
        'hvilka', 'vilka', 'hvardag', 'vardag', 'hvarifrån',
        'varifrån', 'hvarhelst', 'varhelst', 'hvarken', 'varken',
        'hvarmed', 'varmed', 'hvarom', 'varom', 'hvarpå',
        'varpå', 'hvarifrån', 'varifrån', 'hvarvid', 'varvid',
        'hvarest', 'varest', 'hvarjehanda', 'varjehanda', 'hvart',
        'vart', 'hvartannat', 'vartannat', 'hvarannan', 'varannan',
        'hvarvidlag', 'varvidlag', 'hvarenda', 'varenda', 'hvarken',
        'varken', 'hvad', 'vad', 'hvar', 'var',
        'hvarje', 'varje', 'hvem', 'vem', 'hvilka',
        'vilka', 'hvardag', 'vardag', 'hvarifrån', 'varifrån',
        'hvarhelst', 'varhelst', 'hvarken', 'varken', 'hvarmed',
        'varmed', 'hvarom', 'varom', 'hvarpå', 'varpå',
        'hvarifrån', 'varifrån', 'hvarvid', 'varvid', 'hvarest',
        'varest', 'hvarjehanda', 'varjehanda', 'hvart', 'vart',
        'hvartannat', 'vartannat', 'hvarannan', 'varannan', 'hvarvidlag',
        'varvidlag', 'hvarenda', 'varenda', 'hvarken', 'varken',
        'hvad', 'vad', 'hvar', 'var', 'hvarje',
        'varje', 'hvem', 'vem', 'hvilka', 'vilka',
        'hvardag', 'vardag', 'hvarifrån', 'varifrån', 'hvarhelst',
        'varhelst', 'hvarken', 'varken', 'hvarmed', 'varmed',
        'hvarom', 'varom', 'hvarpå', 'varpå', 'hvarifrån',
        'varifrån', 'hvarvid', 'varvid', 'hvarest', 'varest',
        'hvarjehanda', 'varjehanda', 'hvart', 'vart', 'hvartannat',
        'vartannat', 'hvarannan', 'varannan', 'hvarvidlag', 'varvidlag',
        'hvarenda', 'varenda', 'hvarken', 'varken', 'hvad',
        'vad', 'hvar', 'var', 'hvarje', 'varje',
        'hvem', 'vem', 'hvilka', 'vilka', 'hvardag',
        'vardag', 'hvarifrån', 'varifrån', 'hvarhelst', 'varhelst',
        'hvarken', 'varken', 'hvarmed', 'varmed', 'hvarom',
        'varom', 'hvarpå', 'varpå', 'hvarifrån', 'varifrån',
        'hvarvid', 'varvid', 'hvarest', 'varest', 'hvarjehanda',
        'varjehanda', 'hvart', 'vart', 'hvartannat', 'vartannat',
        'hvarannan', 'varannan', 'hvarvidlag', 'varvidlag', 'hvarenda',
        'varenda', 'hvarken', 'varken', 'hvad', 'vad',
        'hvar', 'var', 'hvarje', 'varje', 'hvem',
        'vem', 'hvilka', 'vilka', 'hvardag', 'vardag',
        'hvarifrån', 'varifrån', 'hvarhelst', 'varhelst', 'hvarken',
        'varken', 'hvarmed', 'varmed', 'hvarom', 'varom',
        'hvarpå', 'varpå', 'hvarifrån', 'varifrån', 'hvarvid',
        'varvid', 'hvarest', 'varest', 'hvarjehanda', 'varjehanda',
        'hvart', 'vart', 'hvartannat', 'vartannat', 'hvarannan',
        'varannan', 'hvarvidlag', 'varvidlag', 'hvarenda', 'varenda',
        'hvarken', 'varken', 'hvad', 'vad', 'hvar',
        'var', 'hvarje', 'varje', 'hvem', 'vem',
        'hvilka', 'vilka', 'hvardag', 'vardag', 'hvarifrån',
        'varifrån', 'hvarhelst', 'varhelst', 'hvarken', 'varken',
        'hvarmed', 'varmed', 'hvarom', 'varom', 'hvarpå',
        'varpå', 'hvarifrån', 'varifrån', 'hvarvid', 'varvid',
        'hvarest', 'varest', 'hvarjehanda', 'varjehanda', 'hvart',
        'vart', 'hvartannat', 'vartannat', 'hvarannan', 'varannan',
        'hvarvidlag', 'varvidlag', 'hvarenda', 'varenda', 'hvarken',
        'varken'],
    
    'label': [
        1, 1, 1, 1, 1,
        0, 0, 0, 0, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0, 1, 0, 1, 0,
        1, 0, 1, 0, 1,
        0
       
        
       ]}

print(len(data['label']))
print(len(data['content']))
df = pd.DataFrame(data)
model_kb_classification = AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/evaluation/trained_reform_classification-").to(device)
model_cpt_classification = AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/evaluation/trained_reform_classification061000").to(device)
model_eb_classification = AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/evaluation/trained_reform_classification054000").to(device)
model_spa_classification = AutoModelForSequenceClassification.from_pretrained("/home/laurinemeier/swerick/evaluation/trained_reform_classification258900").to(device)

def evaluate_model(model, tokenizer, df, device):
    inputs = tokenizer(df['content'].tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
    labels = torch.tensor(df['label'].tolist()).to(device)
    outputs = model(**inputs)
    preds = torch.argmax(outputs.logits, axis=1)
    report = classification_report(labels.cpu().numpy(), preds.cpu().numpy(), target_names=['post-réforme', 'pré-réforme'])
    return report

# Exemple d'évaluation (model_base et model_finetuned sont déjà chargés)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
report_base = evaluate_model(model_kb_classification, tokenizer, df, device)
report_cpt = evaluate_model(model_cpt_classification, tokenizer, df, device)
report_eb = evaluate_model(model_eb_classification, exbert_tokenizer, df, device)
report_spa = evaluate_model(model_spa_classification, swerick_tokenizer, df, device)

print("Rapport de classification pour le modèle de base:")
print(report_base)
print("\nRapport de classification pour le modèle finetuné:")
print(report_cpt)
print("\nRapport de classification pour le modèle exbert:")
print(report_eb)
print("\nRapport de classification pour le modèle scratch:")
print(report_spa)

456
456
Rapport de classification pour le modèle de base:
              precision    recall  f1-score   support

post-réforme       0.00      0.00      0.00       228
 pré-réforme       0.50      1.00      0.67       228

    accuracy                           0.50       456
   macro avg       0.25      0.50      0.33       456
weighted avg       0.25      0.50      0.33       456


Rapport de classification pour le modèle finetuné:
              precision    recall  f1-score   support

post-réforme       1.00      0.36      0.53       228
 pré-réforme       0.61      1.00      0.76       228

    accuracy                           0.68       456
   macro avg       0.80      0.68      0.64       456
weighted avg       0.80      0.68      0.64       456


Rapport de classification pour le modèle exbert:
              precision    recall  f1-score   support

post-réforme       1.00      0.27      0.43       228
 pré-réforme       0.58      1.00      0.73       228

    accuracy          

/home/laurinemeier/anaconda3/envs/swerick/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/laurinemeier/anaconda3/envs/swerick/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/laurinemeier/anaconda3/envs/swerick/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [21]:
party_dataset["train"][:10]

{'protocole': ['data/1867/prot-1867--ak--0119.xml',
  'data/1867/prot-1867--ak--0119.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml',
  'data/1867/prot-1867--ak--0121.xml'],
 'Note': ['För min del kan jag ej se något hinder för företagande af dessa val ännu något tidigare än Herr Talmannen föreslagit. Jag har föreställt mig, att vi så mycket som möjligt borde taga vara på den dyrbara tiden, och att nu ifrågakomna val kunde förrättas t. ex. på Tisdagen, och valen till suppleanter i Utskotten på Thorsdagen.',
  'Då tiden snart nalkas det timslag, vid hvilket Kamrarne äro kallade att sig infinna i stadens Storkyrka, hemställer jag huruvida det kan vara skäl att nu utspinna en diskussion, som möjligen kunde blifva alltför vidlyftig att här nu hinna afslutas. Jag a

In [14]:
from evaluation import party_gender_detection,reform_classification,regression_year,party_classifaction_scratch,reform_scratch_classfication
#party_dataset["train"]=load_dataset("csv","swerick_subsetdata_date_train.csv")


for i in range (10):
        train_set = subset_random(party_dataset["train"],100)
        df=train_set.to_pandas()
        df=df.rename(columns={"Note":"content", "gender" : "tag"})

        df.to_csv("swerick_subsetdata_gender_train100.csv")
        reform_scratch_classfication("trained_gender_classification","KBLab/bert-base-swedish-cased","/home/laurinemeier/swerick/exbert-finetuned-imdb/checkpoint-6054000","/home/laurinemeier/swerick/evaluation/swerick_subsetdata_gender_train100.csv",data_path_test="/home/laurinemeier/swerick/evaluation/swerick_subsetdata_gender_test.csv",tokenizer2="/home/laurinemeier/swerick/exbert_tokenizer")

Party alignement classification
training
stdout: 16:04:35 [INFO] (train-bert): Load and save tokenizer...
16:04:36 [INFO] (train-bert): Preprocess datasets...
16:04:36 [INFO] (train-bert): Labels: tensor([0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 1, 0, 0])
16:04:37 [TRAIN] (train-bert): Epoch 0 starts!
16:04:39 [TRAIN] (train-bert): Training Loss: 11.332
16:04:39 [TRAIN] (train-bert): Validation Loss: 10.389
16:04:39 [TRAIN] (train-bert): Validation accuracy: 0.65625
16:04:39 [INFO] (train-bert): Best validation loss so far
16:04:39 [TRAIN] (train-bert): Epoch 1 starts!
16:04:41 [TRAIN] (train-bert): Training Loss: 7.917
16:04:41 [TRAIN] (train-bert): Validation Loss: 11.869
16:04:41 [TRAIN] (train-bert): Vali

In [ ]:
from evaluation import party_gender_detection
with open("comparison_results_gender.txt", "w") :
        pass
for i in [4,6]:
    train_set = subset_random(party_dataset["train"],5000)
    df=train_set.to_pandas()
    df=df.rename(columns={"Note":"content", "gender" : "tag"})
    print()
    df.to_csv("swerick_subsetdata_gender_train.csv")
    party_gender_detection("/home/laurinemeier/swerick/finetuning_hugging_whitespace_lr-finetuned-imdb/checkpoint-858750","swerick_subsetdata_gender_train.csv")


losses_model2, r2_model2 = [], []
losses_model1, r2_model1 = [], []
with open("comparison_results.txt", "r") as file:
    for line in file:
        loss_model1, r2_model1_val, loss_model2, r2_model2_val = line.strip().split(',')
        losses_model1.append(float(loss_model1))
        r2_model1.append(float(r2_model1_val))

        losses_model2.append(float(loss_model2))
        r2_model2.append(float(r2_model2_val))

print(sum(losses_model1)/len(losses_model1))
print(sum(losses_model2)/len(losses_model2))
print(sum(r2_model1)/len(r2_model1))
print(sum(r2_model2)/len(r2_model2))

In [ ]:
# Correlation Perplexity and performance on regression task
train_set = subset_random(party_dataset["train"],1000)
df=train_set.to_pandas()
df=df.rename(columns={"Note":"content", "gender" : "tag"})
df.to_csv("swerick_subsetdata_gender_train.csv")
from evaluation import regression_year, party_gender_detection
with open("comparison_results_gender.txt", "w") :
        pass
checkpoint_directory = "/home/laurinemeier/swerick/finetuning/finetuning_hugging_whitespace-finetuned-imdb"
checkpoint_files = os.listdir(checkpoint_directory)
checkpoint_files.sort(key=lambda x: int(re.search(r'checkpoint-(\d+)', x).group(1)))
selected_checkpoints = [checkpoint_files[i] for i in range(0, len(checkpoint_files), 10)]
for name in selected_checkpoints:
        print(checkpoint_directory + '/' +name)
        party_gender_detection(checkpoint_directory + '/' + name,"swerick_subsetdata_gender_train.csv")


In [ ]:
perplexity=[3.03,2.89,2.807,2.748,2.708,2.674,2.638,2.615,2.591,2.567]
accuracy =[0.7308,0.716,0.7248,0.7012,0.7044,0.7242,0.7104,0.7096,0.7258,0.7064]
f1=[0.71,0.68,0.72,0.71,0.71,0.69,0.71,0.71,0.72,0.71]


plt.figure(figsize=(10, 5))
plt.plot(perplexity,accuracy, marker='o')
#plt.plot(perplexity, r2, label='r2', marker='o')
plt.title(' Accuracy over perplexity')
plt.xlabel('perplexity')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

In [ ]:
losses_model1

In [ ]:

def r2_score(outputs, labels):
    predictions = outputs.logits.squeeze()
    labels_mean = torch.mean(labels.float())
    ss_tot = torch.sum((labels - labels_mean) ** 2)
    ss_res = torch.sum((labels - predictions) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def encode(df, tokenizer):
    # Tokenize all of the sentences and map the tokens to their word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for ix, row in df.iterrows():
        encoded_dict = tokenizer.encode_plus(
            row['content'],
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(df['tag'].tolist())

    return input_ids, attention_masks, labels

In [ ]:
def evaluate(model, loader):
    loss, valid_r2 = 0.0, []
    model.eval()
    for batch in tqdm(loader, total=len(loader)):
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].float().to(device)
        output = model(input_ids,token_type_ids=None,attention_mask=input_mask,labels=labels)
        loss +=output.loss.item()
        r2 = r2_score(output, labels)
        valid_r2.append(r2.item())
        
    r2 = torch.mean(torch.tensor(valid_r2))
    return loss, r2


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd 
from torch.profiler import profile, record_function, ProfilerActivity
model1 = AutoModelForSequenceClassification.from_pretrained(
    "trained/regression_date",
    num_labels=1,
).to(device)

model2 = AutoModelForSequenceClassification.from_pretrained(
    "trained/regression_date_hugging_face",
    num_labels=1).to(device)

df = pd.read_csv("swerick_subsetdata_date_test.csv")
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
input_ids, attention_masks, labels = encode(df, tokenizer)
dataset = TensorDataset(input_ids, attention_masks, labels)
test_loader = DataLoader(
        dataset,
        shuffle=False,
        batch_size=16,
        num_workers=4
    )

loss1,r2=evaluate(model1,test_loader)
loss2,r22=evaluate(model2,test_loader)
            
        


    
    

print("\nLoss model 1:", loss1 * 16/ len(test_loader))
print("\nR2 model1:",torch.mean(torch.tensor(r2)))


print("\nLoss model 2:", loss2* 16 / len(test_loader))
print("\nR2 model2:",torch.mean(torch.tensor(r22)))


In [ ]:
print(prof.key_averages().table(sort_by="cuda_memory_usage", row_limit=10))

In [ ]:
!python3 train_binary_bert.py --data_path "swerick_subsetdata_party_train.csv" --label_names $label_names_str 

In [ ]:
!python3 train_binary_bert.py --model_filename "trained_hugging_face_party_classification" --base_model "finetuning_hugging_whitespace-finetuned-imdb/checkpoint-343500" --data_path "swerick_subsetdata_party_train.csv"